## Test task
Implement a program that synchronizes two folders: source and
replica. The program should maintain a full, identical copy of source
folder at replica folder. Solve the test task by writing a program in
Python.

- Synchronization must be one-way: after the synchronization content of the
  replica folder should be modified to exactly match content of the source
  folder;
  Synchronization should be performed periodically;

- File creation/copying/removal operations should be logged to a file and to the
  console output;
  Folder paths, synchronization interval and log file path should be provided
  using the command line arguments;
  It is undesirable to use third-party libraries that implement folder
  synchronization;
  
- It is allowed (and recommended) to use external libraries implementing other
  well-known algorithms. For example, there is no point in implementing yet
  another function that calculates MD5 if you need it for the task – it is perfectly
  acceptable to use a third-party (or built-in) library.

In [2]:
import os
import hashlib
import time

def calculate_md5(file_path):
    # Calculate the MD5 hash of a file
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

def copy_file(source_file, replica_file):
    # Copy file from source to replica
    with open(source_file, 'rb') as src_file:
        with open(replica_file, 'wb') as dest_file:
            for chunk in iter(lambda: src_file.read(4096), b""):
                dest_file.write(chunk)

source_folder = "../raw/source_folder"
replica_folder = "../replica_folder"
log_file_path = "../.log/sync_log.txt"
sync_interval = 60

# Ensure replica folder exists
if not os.path.exists(replica_folder):
    os.makedirs(replica_folder)

# Track the state of the replica folder
replica_state = {}

while True:
    for root, dirs, files in os.walk(source_folder):
        for file_name in files:
            source_file_path = os.path.join(root, file_name)
            replica_file_path = os.path.join(replica_folder, os.path.relpath(source_file_path, source_folder))

            source_file_md5 = calculate_md5(source_file_path)

            if replica_file_path in replica_state:
                replica_file_md5 = replica_state[replica_file_path]
            else:
                replica_file_md5 = ""

            if source_file_md5 != replica_file_md5:
                # File has changed or is new, copy it
                copy_file(source_file_path, replica_file_path)
                replica_state[replica_file_path] = source_file_md5
                with open(log_file_path, "a") as log_file:
                    log_file.write(f" [{time.now()}] Copied: {source_file_path} -> {replica_file_path}\n")

    # Check for files to delete in replica folder
    for replica_file_path, replica_file_md5 in list(replica_state.items()):
        source_file_path = os.path.join(source_folder, os.path.relpath(replica_file_path, replica_folder))
        if not os.path.exists(source_file_path):
            # File has been deleted in source folder, delete it in replica folder
            os.remove(replica_file_path)
            del replica_state[replica_file_path]
            with open(log_file_path, "a") as log_file:
                log_file.write(f"Deleted: {replica_file_path}\n")

    time.sleep(sync_interval)


KeyboardInterrupt: 

### Running Params

In [1]:
source_folder = "../raw/source_folder"
replica_folder = "../replica_folder"
log_file_path = "../.log/sync_log.txt"
sync_interval = 60

### Running tests

- ``sync_app.py`` is run with the following params:
    - ``source_folder``: [``../raw/source_folder``](../raw/source_folder)
    - ``replica_folder``: [``../replica_folder``](../replica_folder)
    - ``log_file``: [``../.log/sync_log.txt``](../.log/sync_log.txt)
    - ``sync_interval``: 60 seconds

- Blank file ``dummy_text.txt`` is copied to source_folder
- Dummy file ``dummy_text.txt`` is modified in ``source_folder`` by copying the content from ``dummy_text.txt`` in [``../raw/example_files/dummy_text-modify_1.txt``](../raw/example_files/dummy_text-modify_1.txt) which will be repeated again with [``../raw/example_files/dummy_text-modify_2.txt``](../raw/example_files/dummy_text-modify_2.txt).


- It is expected that the file will be copied to [``../replica_folder``](../replica_folder).
- It is expect that the log file will register the operation at [``../.log/sync_log.txt``](../.log/sync_log.txt).

In [ ]:
from time import sleep

!cp ../raw/example_files/dummy_text.txt ../raw/source_folder/

sleep(70)

!cp ../raw/example_files/dummy_text-modify_1.txt ../raw/source_folder/dummy_text.txt

sleep(15)

!cp ../raw/example_files/dummy_text-modify_2.txt ../raw/source_folder/dummy_text.txt

sleep(5)

!cp ../raw/example_files/ratings_matrix.csv ../raw/source_folder/

sleep(90)

!cp ../raw/example_files/ratings_matrix.csv ../raw/source_folder/

sleep(20)

!cp ../raw/example_files/Veeam-Logo.png ../raw/source_folder/

sleep(10)

!rm ../raw/example_files/Veeam-Logo.png ../raw/source_folder/

sleep(70)

!cp ../raw/example_files/Veeam-Logo.png ../raw/source_folder/

### What could be improved:

- Logging could be improved by using a logger object and be appropriatly handled by it since python has a built-in library for that.

- The code could be improved by using a class to handle the syncronization process.

- Testing: While I have tested the code enough to check that it works as expected, I haven't used any testing library to do so due to time constraints.

- A GUI could be implemented to make it more user friendly.

#### Comments by myself:

- While as implementing I have noticed ways to optimize the code, I have decided to leave it as it is to show my thought process and how I have improved the code as I was implementing it.

- This version should be considered as **working as required**, but not as a **final version** which, given more time, I could put more thought in it.

- I have decided to use the ``os`` library to handle the file system operations which have most of the time been used in other projects I have done before and were enough for the job, but I have noticed that there is a library called ``shutil`` which could be used to handle the file system operations in a more efficient way. I have decided to not use it because a version **from scratch** could help me deliver a version as fast as possible due to more familiarity.

- I haven't used any third-party library to handle the MD5 hash, but I have used the built-in library ``hashlib`` to do so.

- I have chosen the Jupyter Notebook to write the code because it is a tool I am familiar which makes it easeir for me to use, showcase and illustrate my thought process.

- I am aware that I could make everything using just linux commands like ``rsync`` or ``dd`` if it were the case and automate the process using cron, but I am more confortable with python code since despite being familiar with, I'm still learning advanced bash.

#### References:

I have used the following references to help me implement the code: 

- Other projects made by myself where I needed to handle file system operations.

- [VERIFY MD5 / SHA256 Hash or Checksum on Linux - File Security (Ubuntu)](https://www.youtube.com/watch?v=uIIn6qVGOJQ) to review some concepts about MD5 hash and test it locally in my system.

- [How To Detect File Changes with Python (and send notification)](https://www.youtube.com/watch?v=lVDajXJEpmg) - is totally different from what I have implemented but it helped me as a first concept.

- ChatGPT also helped me to check how the code would be had I chose to use the ``shutil`` library but I haven't used it nor tested it. Also helped me while debugging.

- [Python Documentation](https://docs.python.org/3/library/os.html) to check os documentation.

- [Python Documentation](https://docs.python.org/3/library/hashlib.html) to check hashlib documentation.


